# Install Taipy

To install Taipy, just `pip install` it.

In [ ]:
%pip install taipy -q

In [ ]:
%pip install pmdarima -q

# Import the packages

In [1]:
from taipy.gui import Gui, Markdown, notify
from taipy import Config, Scope
import taipy as tp

import datetime as dt

from pmdarima import auto_arima

from sklearn.linear_model import LinearRegression

import pandas as pd
import numpy as np

# Taipy Gui Basics
## Markdown Syntax

Taipy uses the Markdown syntax to display elements. `#` creates a title, `*` puts your text in italics and `**` puts it in bold.

![](img/gui_basic_eng.png)

In [2]:
page_md = """
# Taipy

Test **here** to put some *markdown*

Click to access the [doc](https://docs.taipy.io/en/latest/)
"""

In [3]:
Gui(page_md).run(dark_mode=False, run_browser=False)

 * Server starting on http://127.0.0.1:5000


## Visual elements
Create different visual elements. The syntax is always the same for each visual element.  `<|{value}|name_of_visual_element|property_1=value_of_property_1|...|>`
- Create a [slider](https://docs.taipy.io/en/latest/manuals/gui/viselements/slider/) `<|{value}|slider|>`

- Create a [date](https://docs.taipy.io/en/latest/manuals/gui/viselements/date/) `<|{value}|date|>`

- Create a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) `<|{value}|selector|lov={list_of_values}|>`


![](img/control.png)

In [8]:
slider_value = 0
date_value = None
selected_value = None
lov_of_selector = ['Test 1', 'Test 2', 'Test 3']

control_md = """
## Controls

<|{slider_value}|slider|> <|{slider_value}|>

<|{date_value}|date|> <|{date_value}|>

<|{selected_value}|selector|lov={lov_of_selector}|> <|{selected_value}|>
"""

In [9]:
Gui(control_md).run(dark_mode=False, run_browser=False, port=5001)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5001


## Data Viz

A dataset gathering information on the number of deaths, confirmed cases and recovered for different regions is going to be used to create an interactive Dashboard.

In [10]:
path_to_data = "data/covid-19-all.csv"
data = pd.read_csv(path_to_data, low_memory=False)
data[-5:]

,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31
1241951,Zimbabwe,NaN,-19.015438,29.154857,13867.0,11250.0,363.0,2020-12-31


In [11]:
def initialize_case_evolution(data, selected_country='Singapore') -> pd.DataFrame:
    # Aggregation of the dataframe per Country/Region
    country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()
    
    # a country is selected, here India by default
    country_date_df = country_date_df.loc[country_date_df['Country/Region']==selected_country]
    return country_date_df

In [12]:
country_date_df = initialize_case_evolution(data)
country_date_df.head()

C:\Users\jacta\AppData\Local\Temp\ipykernel_26484\1729789157.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()


,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
45216,Singapore,2020-01-23,1.2833,103.8333,1.0,0.0,0.0
45217,Singapore,2020-01-24,1.2833,103.8333,3.0,0.0,0.0
45218,Singapore,2020-01-25,1.2833,103.8333,3.0,0.0,0.0
45219,Singapore,2020-01-26,1.2833,103.8333,4.0,0.0,0.0
45220,Singapore,2020-01-27,1.2833,103.8333,5.0,0.0,0.0


Create a [chart](https://docs.taipy.io/en/latest/manuals/gui/viselements/chart/) showing the evolution of Deaths in Singapore (_Deaths_ for _y_ and _Date_ for _x_). The visual element (chart) has the same syntax as the other ones with specific properties (_x_, _y_, _type_ for example). Here are some [examples of charts](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/charts/bar/). The _x_ and _y_ properties only needs the name of the dataframe columns to display.

![](img/simple_graph.png)

In [15]:
country_md = "<|{country_date_df}|chart|x=Date|y=Deaths|type=bar|>"

In [16]:
Gui(country_md).run(dark_mode=False, run_browser=False, port=5002)

Gui server has been stopped.
 * Server starting on http://127.0.0.1:5002


## Add new traces

- Add on the graph the number of Confirmed and Recovered cases (_Confirmed_ and _Recovered_) with the number of Deaths
- _y_ (and _x_) can be indexed this way to add more traces (`y[1]=`, `y[2]=`, `y[3]=`).

![](img/multi_traces.png)

In [17]:
country_md = "<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|>"

In [18]:
Gui(country_md).run(dark_mode=False, run_browser=False, port=5003)

 * Server starting on http://127.0.0.1:5003


## Style the graph with personalized properties
The _layout_ dictionnary specifies how bars should be displayed. They would be 'stacked'.

The _options_ dictionary will change the opacity of the unselected markers.

These are Plotly properties.

![](img/stack_chart.png)

In [19]:
layout = {'barmode':'stack'}
options = {"unselected":{"marker":{"opacity":0.5}}}
country_md = "<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>"

In [20]:
Gui(country_md).run(dark_mode=False, run_browser=False, port=5004)

 * Server starting on http://127.0.0.1:5004


## Add texts that sums up the data

Use the [text](https://docs.taipy.io/en/latest/manuals/gui/viselements/text/) visual element.

- Add the total number of Deaths (last line of _country_date_df_)
- Add the total number of Recovered (last line of _country_date_df_)
- Add the total number of Confirmed (last line of _country_date_df_)


In [21]:
country_date_df

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
45216,Singapore,2020-01-23,1.2833,103.8333,1.0,0.0,0.0
45217,Singapore,2020-01-24,1.2833,103.8333,3.0,0.0,0.0
45218,Singapore,2020-01-25,1.2833,103.8333,3.0,0.0,0.0
45219,Singapore,2020-01-26,1.2833,103.8333,4.0,0.0,0.0
45220,Singapore,2020-01-27,1.2833,103.8333,5.0,0.0,0.0
...,...,...,...,...,...,...,...
45555,Singapore,2020-12-27,1.2833,103.8333,58524.0,58370.0,29.0
45556,Singapore,2020-12-28,1.2833,103.8333,58529.0,58386.0,29.0
45557,Singapore,2020-12-29,1.2833,103.8333,58542.0,58400.0,29.0
45558,Singapore,2020-12-30,1.2833,103.8333,58569.0,58411.0,29.0


This is how we can get the total number of Deaths from the dataset for Singapore.

In [22]:
country_date_df.iloc[-1, 6] # gives the number of deaths for Singapore (5 is for recovered and 4 is confirmed)

29.0

Use the [text](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/text/) visual element. Note that between `{}`, any Python variable can be put but also any Python code.

![](img/control_text.png)

In [24]:
country_md = """
## Deaths <|{country_date_df.iloc[-1, 6]}|text|>

## Recovered <|{country_date_df.iloc[-1, 5]}|text|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|text|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [25]:
Gui(country_md).run(dark_mode=False, run_browser=False, port=5005)

 * Server starting on http://127.0.0.1:5005


## Local _on_change_

- Add a [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) with `dropdown=True` containing the name of all the _Country/region_
- Give to the _on_change_ selector property the name of the _on_change_country_ function. This function will be called when the selector will be used.
- This function has a 'state' parameter and has to be completed. When the selector is used, this function is called with the _state_ argument. It contains all the Gui variables; 'state.country_date_df' is then the dataframe used in the Gui.

![](img/on_change_local.png)

In [26]:
country_lov = sorted(data["Country/Region"].dropna().unique().tolist())
selected_country = "Singapore"

country_md = """
<|{selected_country}|selector|lov={country_lov}|dropdown|on_change=on_change_country|>
        

## Deaths <|{country_date_df.iloc[-1, 6]}|>

## Recovered <|{country_date_df.iloc[-1, 5]}|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [27]:
def on_change_country(state):
    # state contains all the Gui variables and this is through this state variable that we can update the Gui
    # state.selected_country, state.country_date_df, ...
    # update country_date_df with the right country (use initialize_case_evolution)
    print("Chosen country: ", state.selected_country)
    state.country_date_df = initialize_case_evolution(data, state.selected_country)

In [28]:
Gui(country_md).run(dark_mode=False, run_browser=False, port=5006)

 * Server starting on http://127.0.0.1:5006
Chosen country:  Poland


C:\Users\jacta\AppData\Local\Temp\ipykernel_26484\1729789157.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()


Chosen country:  New Zealand


C:\Users\jacta\AppData\Local\Temp\ipykernel_26484\1729789157.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()


Chosen country:  Moldova


C:\Users\jacta\AppData\Local\Temp\ipykernel_26484\1729789157.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()


Chosen country:  Luxembourg


C:\Users\jacta\AppData\Local\Temp\ipykernel_26484\1729789157.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()


## Layout

Use the [layout](https://docs.taipy.io/en/latest/manuals/gui/viselements/layout/) block to change the page structure. This block creates invisible columns to put text/visual elements in.

Syntax :
```
<|layout|columns=1 1 1 ...|
(first column)

(in second column)

(third column)
(again, third column)

(...)
|>
```

In [29]:
final_country_md = """
<|layout|columns=1 1 1 1|
<|{selected_country}|selector|lov={country_lov}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{country_date_df.iloc[-1, 6]}|>

## Recovered <|{country_date_df.iloc[-1, 5]}|>

## Confirmed <|{country_date_df.iloc[-1, 4]}|>
|>

<|{country_date_df}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [30]:
Gui(final_country_md).run(dark_mode=False, run_browser=False, port=5007)

 * Server starting on http://127.0.0.1:5007
Chosen country:  Saint Martin


C:\Users\jacta\AppData\Local\Temp\ipykernel_26484\1729789157.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()


Chosen country:  Portugal


C:\Users\jacta\AppData\Local\Temp\ipykernel_26484\1729789157.py:3: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  country_date_df = data.groupby(["Country/Region",'Date']).sum().reset_index()


![](img/layout.png)

# Map

In [31]:
def initialize_map(data):
    data['Province/State'] = data['Province/State'].fillna(data["Country/Region"])
    data_province = data.groupby(["Country/Region",
                                  'Province/State',
                                  'Longitude',
                                  'Latitude'])\
                         .max()

    data_province_displayed = data_province[data_province['Deaths']>10].reset_index()

    data_province_displayed['Size'] = np.sqrt(data_province_displayed.loc[:,'Deaths']/data_province_displayed.loc[:,'Deaths'].max())*80 + 3
    data_province_displayed['Text'] = data_province_displayed.loc[:,'Deaths'].astype(str) + ' deaths</br>' + data_province_displayed.loc[:,'Province/State']
    return data_province_displayed

In [32]:
data_province_displayed = initialize_map(data)
data_province_displayed.head()

,Country/Region,Province/State,Longitude,Latitude,Confirmed,Recovered,Deaths,Date,Size,Text
0,Afghanistan,Afghanistan,67.709953,33.93911,51526.0,41727.0,2191.0,2020-12-31,17.771247,2191.0 deaths</br>Afghanistan
1,Albania,Albania,20.168300,41.15330,58316.0,33634.0,1181.0,2020-12-31,13.844784,1181.0 deaths</br>Albania
2,Algeria,Algeria,1.659600,28.03390,99610.0,67127.0,2756.0,2020-12-31,19.566684,2756.0 deaths</br>Algeria
3,Andorra,Andorra,1.521800,42.50630,8049.0,7432.0,84.0,2020-12-31,5.892249,84.0 deaths</br>Andorra
4,Angola,Angola,17.873900,-11.20270,17553.0,11044.0,405.0,2020-12-31,9.350728,405.0 deaths</br>Angola


Properties to style the map
- marker color corresponds to the number of Deaths (column _Deaths_)
- marker sizes corresponds to the size in _Size_ column which is created from the number of Deaths

layout_map permet defined the initial zoom and position of the map


In [33]:
marker_map = {"color":"Deaths", "size": "Size", "showscale":True, "colorscale":"Viridis"}
layout_map = {
            "dragmode": "zoom",
            "mapbox": { "style": "open-street-map", "center": { "lat": 38, "lon": -90 }, "zoom": 3}
            }

We give to Plotly:
- a map type
- the name of the latitude column
- the name of the longitude column
- properties: on the size and color of the markers
- the name of the column for the text of the points

In [34]:
selected_points = []
map_md = """
<|{data_province_displayed}|chart|type=scattermapbox|lat=Latitude|lon=Longitude|marker={marker_map}|layout={layout_map}|text=Text|mode=markers|height=800px|options={options}|>
"""

In [35]:
Gui(map_md).run(dark_mode=False, run_browser=False, port=5008)

 * Server starting on http://127.0.0.1:5008


![](img/carte.png)

# Part and the _render_ property
- Create a [toggle](https://docs.taipy.io/en/latest/manuals/gui/viselements/toggle/) (works the same as a selector) with a lov of 'Map' an 'Country'
- Create two part blocks that renders or not depending on the value of the toggle
    - To do this, use the fact that in the _render_ property of the part block, Python code can be inserted in `{}`

In [36]:
representation_toggle = ['Map', 'Country']
selected_representation = representation_toggle[0]

In [37]:
main_page = """
<|{selected_representation}|toggle|lov={representation_toggle}|>

<|part|render={selected_representation == "Country"}|
"""+final_country_md+"""
|>

<|part|render={selected_representation == "Map"}|
"""+map_md+"""
|>
""" 

In [38]:
Gui(main_page).run(dark_mode=False, run_browser=False, port=5009)

 * Server starting on http://127.0.0.1:5009


![](img/part_render.png)

# Taipy Core
Here are the functions that we are going to use to predict the number of Deaths for a country.
We will:
- preprocess the data (_preprocess_),
- create a training and testing database (_make_train_test_data_),
- train a model (_train_model_),
- generate predictions (_forecast_),
- generate a dataframe with the historical data and the predictions (_result_)

![](img/all_architecture.svg)

In [39]:
# initialise variables
selected_scenario = None
scenario_selector = None

first_date = dt.datetime(2020,11,1)

scenario_name = None

result = None

In [ ]:
#Config.configure_job_executions(mode="standalone", nb_of_workers=2)

In [41]:
def add_features(data):
    dates = pd.to_datetime(data["Date"])
    data["Months"] = dates.dt.month
    data["Days"] = dates.dt.isocalendar().day
    data["Week"] = dates.dt.isocalendar().week
    data["Day of week"] = dates.dt.dayofweek
    return data

def create_train_data(final_data, date):
    bool_index = pd.to_datetime(final_data['Date']) <= date
    train_data = final_data[bool_index]
    return train_data

def preprocess(initial_data, country, date):
    data = initial_data.groupby(["Country/Region",'Date'])\
                       .sum()\
                       .dropna()\
                       .reset_index()

    final_data = data.loc[data['Country/Region']==country].reset_index(drop=True)
    final_data = final_data[['Date','Deaths']]
    final_data = add_features(final_data)
    
    train_data = create_train_data(final_data, date)
    return final_data, train_data


def train_arima(train_data):
    model = auto_arima(train_data['Deaths'],
                       start_p=1, start_q=1,
                       max_p=5, max_q=5,
                       start_P=0, seasonal=False,
                       d=1, D=1, trace=True,
                       error_action='ignore',  
                       suppress_warnings=True)
    model.fit(train_data['Deaths'])
    return model


def forecast(model):
    predictions = model.predict(n_periods=60)
    return predictions


def result(final_data, predictions, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(len(predictions))])
    final_data['Date'] = pd.to_datetime(final_data['Date'])
    predictions = pd.concat([pd.Series(dates, name="Date"),
                             pd.Series(predictions, name="Predictions")], axis=1)
    return final_data.merge(predictions, on="Date", how="outer")

First we must define the Data Nodes then the tasks (associated to the Python function). Furthermore, we gather these tasks into different pipelines and these pipelines into a scenario.

A Data Node needs a **unique id**. If needed, the storage type can be changed for CSV and SQL. Other parameters are then needed.

### Data Nodes and Task for preprocess

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [42]:
initial_data_cfg = Config.configure_data_node(id="initial_data",
                                              storage_type="csv",
                                              path=path_to_data,
                                              cacheable=True,
                                              validity_period=dt.timedelta(days=5),
                                              scope=Scope.GLOBAL)

country_cfg = Config.configure_data_node(id="country", default_data="India",
                                         cacheable=True, validity_period=dt.timedelta(days=5))


date_cfg = Config.configure_data_node(id="date", default_data=dt.datetime(2020,10,10),
                                         cacheable=True, validity_period=dt.timedelta(days=5))

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [43]:
final_data_cfg =  Config.configure_data_node(id="final_data",
                                            cacheable=True, validity_period=dt.timedelta(days=5))


train_data_cfg =  Config.configure_data_node(id="train_data", cacheable=True, validity_period=dt.timedelta(days=5))


<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [44]:
task_preprocess_cfg = Config.configure_task(id="task_preprocess_data",
                                           function=preprocess,
                                           input=[initial_data_cfg, country_cfg, date_cfg],
                                           output=[final_data_cfg,train_data_cfg])

### Data Nodes and Task for train_model

<img src="img/train_model.svg" alt="drawing" width="500"/>

In [45]:
model_cfg = Config.configure_data_node(id="model", cacheable=True, validity_period=dt.timedelta(days=5), scope=Scope.PIPELINE)

task_train_cfg = Config.configure_task(id="task_train",
                                      function=train_arima,
                                      input=train_data_cfg,
                                      output=model_cfg)

### Data Nodes and Task for forecast

<img src="img/forecast_arima.svg" alt="drawing" width="500"/>

In [46]:
predictions_cfg = Config.configure_data_node(id="predictions", scope=Scope.PIPELINE)

task_forecast_cfg = Config.configure_task(id="task_forecast",
                                      function=forecast,
                                      input=model_cfg,
                                      output=predictions_cfg)

### Data Nodes and Task for result

<img src="img/result.svg" alt="drawing" width="500"/>

In [47]:
# TO COMPLETE
result_cfg = Config.configure_data_node(id="result", scope=Scope.PIPELINE)

# input order: final_data, predictions, date
task_result_cfg = Config.configure_task(id="task_result",
                                      function=result,
                                      input=[final_data_cfg, predictions_cfg, date_cfg],
                                      output=result_cfg)

## [Configuration of pipelines](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_pipeline)

In [48]:
pipeline_preprocessing_cfg = Config.configure_pipeline(id="pipeline_preprocessing",
                                                       task_configs=[task_preprocess_cfg])

pipeline_arima_cfg = Config.configure_pipeline(id="ARIMA",
                                               task_configs=[task_train_cfg,
                                                             task_forecast_cfg,
                                                             task_result_cfg])

## Add more models

<img src="img/pipeline_linear_regression.svg" alt="drawing" width="500"/>

In [50]:
def train_linear_regression(train_data):    
    y = train_data['Deaths']
    X = train_data.drop(['Deaths','Date'], axis=1)
    
    model = LinearRegression()
    model.fit(X,y)
    return model

def forecast_linear_regression(model, date):
    dates = pd.to_datetime([date + dt.timedelta(days=i)
                            for i in range(60)])
    X = add_features(pd.DataFrame({"Date":dates}))
    X.drop('Date', axis=1, inplace=True)
    predictions = model.predict(X)
    return predictions


task_train_cfg = Config.configure_task(id="task_train",
                                      function=train_linear_regression,
                                      input=train_data_cfg,
                                      output=model_cfg)

task_forecast_cfg = Config.configure_task(id="task_forecast",
                                      function=forecast_linear_regression,
                                      input=[model_cfg, date_cfg],
                                      output=predictions_cfg)

pipeline_linear_regression_cfg = Config.configure_pipeline(id="LinearRegression",
                                               task_configs=[task_train_cfg,
                                                             task_forecast_cfg,
                                                             task_result_cfg])

## [Configuration of scénario](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_scenario)

In [51]:
scenario_cfg = Config.configure_scenario(id='scenario', pipeline_configs=[pipeline_preprocessing_cfg,
                                                                          pipeline_arima_cfg,
                                                                          pipeline_linear_regression_cfg])

## Creation and submit of scenario

In [52]:
scenario = tp.create_scenario(scenario_cfg, name='First Scenario')
tp.submit(scenario)

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-10-15 17:23:36,339][Taipy][INFO] job JOB_task_preprocess_data_bd2564ac-a65c-4299-9850-898c0d9899e3 is completed.


C:\Users\jacta\AppData\Local\Temp\ipykernel_26484\2389702588.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()\


[2022-10-15 17:23:36,685][Taipy][INFO] job JOB_task_train_db9043ca-7d7d-40d0-9908-d6bd2c0ed1e4 is completed.
[2022-10-15 17:23:36,775][Taipy][INFO] job JOB_task_forecast_d0a96f60-0c8d-4261-afd3-9112c79659d3 is completed.
[2022-10-15 17:23:36,844][Taipy][INFO] job JOB_task_result_9005b3b5-7663-4cf4-909d-0437f9769531 is completed.
[2022-10-15 17:23:36,994][Taipy][INFO] job JOB_task_train_c3a84d37-780b-439c-842e-baae276b3f2c is completed.
[2022-10-15 17:23:37,064][Taipy][INFO] job JOB_task_forecast_bf18944c-1291-46fb-961b-aeebac462736 is completed.
[2022-10-15 17:23:37,117][Taipy][INFO] job JOB_task_result_ba2603fc-d7f5-46a9-9bea-c6512596c8fd is completed.


{'PIPELINE_pipeline_preprocessing_aaf0c203-e2fe-4f8b-b87d-e70da85d8aa4': [<taipy.core.job.job.Job at 0x2b7cd3caf20>],
 'PIPELINE_ARIMA_9c07fe3f-2cc8-4288-9ecc-ab8f2c475fee': [<taipy.core.job.job.Job at 0x2b7cd3c8d60>,
 'PIPELINE_LinearRegression_9e386d43-a309-4e3c-a42b-ab19ccbc8fa7': [<taipy.core.job.job.Job at 0x2b7cb587880>,
  <taipy.core.job.job.Job at 0x2b7cd3ca050>]}

In [53]:
scenario.initial_data.read()

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
0,NaN,NaN,NaN,NaN,51526.0,41727.0,2191.0,2021-01-01
1,NaN,NaN,NaN,NaN,58316.0,33634.0,1181.0,2021-01-01
2,NaN,NaN,NaN,NaN,99897.0,67395.0,2762.0,2021-01-01
3,NaN,NaN,NaN,NaN,8117.0,7463.0,84.0,2021-01-01
4,NaN,NaN,NaN,NaN,17568.0,11146.0,405.0,2021-01-01
...,...,...,...,...,...,...,...,...
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31


In [54]:
scenario.train_data.read()

,Date,Deaths,Months,Days,Week,Day of week
0,2020-01-30,0.0,1,4,5,3
1,2020-01-31,0.0,1,5,5,4
2,2020-02-01,0.0,2,6,5,5
3,2020-02-02,0.0,2,7,5,6
4,2020-02-03,0.0,2,1,6,0
...,...,...,...,...,...,...
250,2020-10-06,104555.0,10,2,41,1
251,2020-10-07,105526.0,10,3,41,2
252,2020-10-08,106490.0,10,4,41,3
253,2020-10-09,107416.0,10,5,41,4


In [55]:
scenario.LinearRegression.predictions.read()

array([74825.01627879, 75240.33209199, 75021.90955514, 75437.22536835,
       75852.54118155, 76267.85699475, 76683.17280796, 77098.48862116,
       77513.80443437, 77295.38189751, 77710.69771072, 78126.01352392,
       78541.32933712, 78956.64515033, 79371.96096353, 79787.27677674,
       79568.85423988, 79984.17005309, 80399.48586629, 80814.8016795 ,
       81230.1174927 , 81645.4333059 , 84018.0020429 , 83799.57950605,
       84214.89531925, 84630.21113246, 85045.52694566, 85460.84275886,
       85876.15857207, 86291.47438527, 86073.05184842, 86488.36766162,
       86903.68347483, 87318.99928803, 87734.31510124, 88149.63091444,
       88564.94672764, 88346.52419079, 88761.84000399, 89177.1558172 ,
       89592.4716304 , 90007.78744361, 90423.10325681, 90838.41907001,
       90619.99653316, 91035.31234636, 91450.62815957, 91865.94397277,
       92281.25978598, 92696.57559918, 93111.89141238, 92893.46887553,
       95266.03761253, 95681.35342573, 96096.66923894, 96511.98505214,
      

## Caching
Some job are skipped because no change has been done to the "input" Data Nodes.

In [56]:
tp.submit(scenario)

[2022-10-15 17:25:12,990][Taipy][INFO] job JOB_task_preprocess_data_dca7dbbe-9356-42f6-8c1c-a6a4358fb05f is skipped.
[2022-10-15 17:25:13,119][Taipy][INFO] job JOB_task_train_d07578a0-652b-4798-888e-32b11b422415 is skipped.
[2022-10-15 17:25:13,188][Taipy][INFO] job JOB_task_forecast_0a150961-805c-40b4-b39c-28a401e26390 is completed.
[2022-10-15 17:25:13,241][Taipy][INFO] job JOB_task_result_42722c47-6fe7-4b57-80df-694e56d2c400 is completed.
[2022-10-15 17:25:13,442][Taipy][INFO] job JOB_task_train_2386c92f-9d4a-4920-9604-e60742d26a1a is skipped.
[2022-10-15 17:25:13,529][Taipy][INFO] job JOB_task_forecast_c58e7fb7-234e-4d5c-8471-31c73ec95b14 is completed.
[2022-10-15 17:25:13,582][Taipy][INFO] job JOB_task_result_0608213c-4fbb-4f7d-a4e9-37a9377e4705 is completed.


{'PIPELINE_pipeline_preprocessing_aaf0c203-e2fe-4f8b-b87d-e70da85d8aa4': [<taipy.core.job.job.Job at 0x2b7d8c60b20>],
 'PIPELINE_ARIMA_9c07fe3f-2cc8-4288-9ecc-ab8f2c475fee': [<taipy.core.job.job.Job at 0x2b7cb64f100>,
 'PIPELINE_LinearRegression_9e386d43-a309-4e3c-a42b-ab19ccbc8fa7': [<taipy.core.job.job.Job at 0x2b7dd93e650>,
  <taipy.core.job.job.Job at 0x2b7dd93e740>]}

## Write in data nodes

To write a data node:

`<Data Node>.write(new_value)`

In [57]:
scenario.country.write('US')
tp.submit(scenario)
scenario.result.read()

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-10-15 17:25:40,017][Taipy][INFO] job JOB_task_preprocess_data_2deb484a-873a-446d-8450-36982f7f0bb9 is completed.


C:\Users\jacta\AppData\Local\Temp\ipykernel_26484\2389702588.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()\


[2022-10-15 17:25:40,360][Taipy][INFO] job JOB_task_train_69ad6cb7-64fc-44d8-a543-87ed250fd98a is completed.
[2022-10-15 17:25:40,540][Taipy][INFO] job JOB_task_forecast_3e445841-f4cb-4998-ad5f-1cdbbcededa3 is completed.
[2022-10-15 17:25:40,630][Taipy][INFO] job JOB_task_result_3a7f3a61-e6c7-4432-a13f-01421d6fff94 is completed.
[2022-10-15 17:25:40,848][Taipy][INFO] job JOB_task_train_c1da3033-ecde-4c21-a1bb-53feaca07a50 is completed.
[2022-10-15 17:25:40,935][Taipy][INFO] job JOB_task_forecast_3fb7dfa5-dd62-4fd3-b4af-446c76a7d511 is completed.
[2022-10-15 17:25:40,992][Taipy][INFO] job JOB_task_result_b8178ccd-8c6d-4a9e-92e7-d254968c9263 is completed.


,Date,Deaths,Months,Days,Week,Day of week,Predictions
0,2020-01-22,0.0,1,3,4,2,NaN
1,2020-01-23,0.0,1,4,4,3,NaN
2,2020-01-24,0.0,1,5,4,4,NaN
3,2020-01-25,0.0,1,6,4,5,NaN
4,2020-01-26,0.0,1,7,4,6,NaN
...,...,...,...,...,...,...,...
340,2020-12-27,334533.0,12,7,52,6,NaN
341,2020-12-28,336438.0,12,1,53,0,NaN
342,2020-12-29,340061.0,12,2,53,1,NaN
343,2020-12-30,343783.0,12,3,53,2,NaN


## Simple framework

In [58]:
scenario = tp.create_scenario(scenario_cfg, name='Second Scenario')
tp.submit(scenario)

C:\Users\jacta\AppData\Local\R-MINI~1\envs\notebook\lib\site-packages\taipy\core\data\csv.py:161: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-10-15 17:25:49,334][Taipy][INFO] job JOB_task_preprocess_data_f6ecff06-2ba4-44e0-838f-8ae363e2d16b is completed.


C:\Users\jacta\AppData\Local\Temp\ipykernel_26484\2389702588.py:16: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  .sum()\


[2022-10-15 17:25:49,781][Taipy][INFO] job JOB_task_train_9e79b1e5-5e84-48ef-8eb1-442bf04b10f4 is completed.
[2022-10-15 17:25:49,877][Taipy][INFO] job JOB_task_forecast_f1795117-7f3a-46ec-b956-f394ea3a958a is completed.
[2022-10-15 17:25:49,950][Taipy][INFO] job JOB_task_result_6ef33b27-4e4d-428e-949a-d174c0d3f773 is completed.
[2022-10-15 17:25:50,255][Taipy][INFO] job JOB_task_train_37fe914a-d639-4e5e-b456-584fd74e0154 is completed.
[2022-10-15 17:25:50,366][Taipy][INFO] job JOB_task_forecast_6c4b081a-4c6e-4ca7-8fe5-6cff6a4f1545 is completed.
[2022-10-15 17:25:50,418][Taipy][INFO] job JOB_task_result_fcac2099-1fd2-4dda-8d31-22db69eadd92 is completed.


{'PIPELINE_pipeline_preprocessing_3c2c4f2a-50b8-48a0-9b25-ae5294373a7f': [<taipy.core.job.job.Job at 0x2b7df3de0b0>],
 'PIPELINE_ARIMA_f14be787-fb59-4d05-87f9-917baa81551d': [<taipy.core.job.job.Job at 0x2b7dd95b4c0>,
 'PIPELINE_LinearRegression_243bc7fc-86dc-43ff-8bac-0df0175fc44d': [<taipy.core.job.job.Job at 0x2b7dd93db40>,
  <taipy.core.job.job.Job at 0x2b7e3611090>]}

In [59]:
scenario.ARIMA.model.read()

LinearRegression()

In [60]:
scenario.pipelines['LinearRegression'].model.read()

LinearRegression()

In [61]:
[s.country.read() for s in tp.get_scenarios()]

['India', 'US']

In [62]:
[s.date.read() for s in tp.get_scenarios()]

[datetime.datetime(2020, 10, 10, 0, 0), datetime.datetime(2020, 10, 10, 0, 0)]

## Create a Gui for the backend
_scenario_selector_ lets you choose a scenario and display its results.

In [63]:
scenario_selector = [(s.id, s.name) for s in tp.get_scenarios()]
selected_scenario = scenario.id
print(scenario_selector,'\n', selected_scenario)

[('SCENARIO_scenario_a18a0b01-9adf-4c1c-9903-c72a3f339e3b', 'Second Scenario'), ('SCENARIO_scenario_df6847a3-7024-4c60-a718-a9a172f872b4', 'First Scenario')] 
 SCENARIO_scenario_a18a0b01-9adf-4c1c-9903-c72a3f339e3b


In [64]:
result_arima = scenario.pipelines['ARIMA'].result.read()
result_rd = scenario.pipelines['LinearRegression'].result.read()
result = result_rd.merge(result_arima, on="Date", how="outer").sort_values(by='Date')
result

,Date,Deaths_x,Months_x,Days_x,Week_x,Day of week_x,Predictions_x,Deaths_y,Months_y,Days_y,Week_y,Day of week_y,Predictions_y
0,2020-01-30,0.0,1,4,5,3,NaN,0.0,1,4,5,3,NaN
1,2020-01-31,0.0,1,5,5,4,NaN,0.0,1,5,5,4,NaN
2,2020-02-01,0.0,2,6,5,5,NaN,0.0,2,6,5,5,NaN
3,2020-02-02,0.0,2,7,5,6,NaN,0.0,2,7,5,6,NaN
4,2020-02-03,0.0,2,1,6,0,NaN,0.0,2,1,6,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,2020-12-27,147901.0,12,7,52,6,NaN,147901.0,12,7,52,6,NaN
333,2020-12-28,148153.0,12,1,53,0,NaN,148153.0,12,1,53,0,NaN
334,2020-12-29,148439.0,12,2,53,1,NaN,148439.0,12,2,53,1,NaN
335,2020-12-30,148738.0,12,3,53,2,NaN,148738.0,12,3,53,2,NaN


**Tips** : the _value_by_id_ property if set to True for a selected will make _selected_scenario_ directly refer to the first element of the tupple (here the id)

![](img/predictions.png)

In [66]:
prediction_md = """
<|layout|columns=1 2 5 1 3|
<|{scenario_name}|input|label=Name|>

<br/>
<|Create|button|on_action=create_new_scenario|>

Prediction date
<|{first_date}|date|>

<|{selected_country}|selector|lov={country_lov}|dropdown|on_change=on_change_country|label=Country|>

<br/>
<|Submit|button|on_action=submit_scenario|>

<|{selected_scenario}|selector|lov={scenario_selector}|on_change=actualize_graph|dropdown|value_by_id|label=Scenario|>
|>

<|{result}|chart|x=Date|y[1]=Deaths_x|type[1]=bar|y[2]=Predictions_x|y[3]=Predictions_y|>
"""

In [67]:
def create_new_scenario(state):
    scenario = tp.create_scenario(scenario_cfg, name=state.scenario_name)
    state.scenario_selector += [(scenario.id, scenario.name)]

In [68]:
def submit_scenario(state):
    # 1) get the selected scenario
    # 2) write in country Data Node, the selected country
    # 3) submit the scenario
    # 4) actualize le graph avec actualize_graph
    scenario = tp.get(state.selected_scenario)
    scenario.country.write(state.selected_country)
    tp.submit(scenario)
    actualize_graph()
    ...

In [69]:
def actualize_graph(state):
    scenario = tp.get(state.selected_scenario)
    result_arima = scenario.pipelines['ARIMA'].result.read()
    result_rd = scenario.pipelines['LinearRegression'].result.read()
    state.result = result_rd.merge(result_arima, on="Date", how="outer").sort_values(by='Date')
    # 1) update the result dataframe if result is not None
    # 2) change selected_country with the predicted country of the scenario

In [ ]:
Gui(prediction_md).run(dark_mode=False, port=5010)

# Multi-pages and Taipy Rest

To create a multi-pages app, we only need a dictionary with names as the keys and the Markdowns as the values.

The _navbar_ control (<|navbar|>) has a default behaviour. It redirects to the different pages of the app automatically. Other solutions exists.

![](img/multi_pages.png)

In [70]:
navbar_md = "<center>\n<|navbar|>\n</center>"

pages = {
    "Map":navbar_md+map_md,
    "Country":navbar_md+final_country_md,
    "Predictions":navbar_md+prediction_md
}

rest = tp.Rest()

gui_multi_pages = Gui(pages=pages)
tp.run(gui_multi_pages, rest, dark_mode=False, port=5011)

 * Server starting on http://127.0.0.1:5011
